---
# Chapter 7
## Function Decorators and Closures

---

## Decorators 101

---
### Example 7-1: A decorator usually repalces a function with a different one

In [83]:
def deco(func):
    def inner():
        print('running inner()')
    return inner


@deco
def target():
    print('running target()')


target()

running inner()


In [84]:
target

<function __main__.deco.<locals>.inner()>

## When python Executes Decorators

In [85]:
# See code in "registration.py" file

import registration

In [86]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

## Decorator-Enhaced Strategy Pattern

---
### Example 7-3: The _promos_ list is filled with _promotion_ decorator

In [87]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 10:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.7
    return 0


## Variable Scope Rules

---
### Example 7-4: Function reading a local variable and a global varibale

In [88]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [ ]:
b = 6
f1(3)

3
6


---
### Example 7-5: Varibale b is local, because it is assigned a value in the body of the function.

In [ ]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 4

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [ ]:
b = 6
def f2(a):
    global b
    print(a)
    print(b)
    b = 9

f2(3)
print(b)

3
6
9


In [ ]:
b

9

---
### Example 7-6: Disassembly of the f1 function from Example 7-4

In [ ]:
from dis import dis
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


---
### Example 7-7: Disassembly od the f2 function from Example 7-5

In [ ]:
dis(f2)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (9)
             18 STORE_GLOBAL             1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## Closures

---
### Example 7-8: average_oo.py: A class to calculate a running average

In [ ]:
# average_oo.py

class Average():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)


avg = Average()

print("avg(10) =", avg(10))
print("avg(11) =", avg(11))
print("avg(12) =", avg(12))

avg(10) = 10.0
avg(11) = 10.5
avg(12) = 11.0


---
### Example 7-9: averate.py: A higher-order function to calculate a running average

In [ ]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)

    return averager

---
### Example 7-10: Testing example 7-9

In [ ]:
avg = make_averager()
print("avg(10)", avg(10))
print("avg(11)", avg(11))
print("avg(12)", avg(12))

avg(10) 10.0
avg(11) 10.5
avg(12) 11.0


---
### Example 7-11:  Inspecting the function created by *make_averager* in example 7-9.

In [ ]:
print("{0:>25}: {1}".format("avg.__code__", avg.__code__))
print("{0:>25}: {1}".format("avg.__code__.co_varnames", avg.__code__.co_varnames))
print("{0:>25}: {1}".format("avg.__code__.co_freevar", avg.__code__.co_freevars))
print("{0:>25}: {1}".format("avg.__closure__", avg.__closure__))
print("{0:>25}: {1}".format("avg.__closure__[0].cell_contents", avg.__closure__[0].cell_contents))


             avg.__code__: <code object averager at 0x102c9f9d0, file "/var/folders/lc/kh6gj3b90s5bnw_nd9llbh540000gn/T/ipykernel_17998/268262257.py", line 4>
 avg.__code__.co_varnames: ('new_value', 'total')
  avg.__code__.co_freevar: ('series',)
          avg.__closure__: (<cell at 0x10528ebb0: list object at 0x10be67240>,)
avg.__closure__[0].cell_contents: [10, 11, 12]


In [ ]:
dir(avg.__closure__)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'count',
 'index']

## The nonlocal Declaration

---
### Example 7-13: A broken higher-order function to calculate a running *average* without keeping all history

In [ ]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager


In [ ]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

---
### Example 7-14: Calculate a running average without keeping all history (fixed with the use of nonlocal)

In [ ]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

In [ ]:
avg = make_averager()
avg(10)

10.0

## Implementing a Simple Decorator

---
### Example 7-15: A simple decorator to output the running time of fucntion

In [ ]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

---
### Example 7-16: Using the clock decorator

In [ ]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)


print("*"*40, "calling snooze(.123)")
snooze(.123)
print("*"*40, "calling factorial(6)")
factorial(6)

**************************************** calling snooze(.123)
[0.12739511s] snooze(0.123) -> None
**************************************** calling factorial(6)
[0.00000112s] factorial(1) -> 1
[0.00011274s] factorial(2) -> 2
[0.00022305s] factorial(3) -> 6
[0.00028195s] factorial(4) -> 24
[0.00030777s] factorial(5) -> 120
[0.00033142s] factorial(6) -> 720


720

**************************************** calling snooze(.123)
[0.12780315s] snooze(0.123) -> None
**************************************** calling factorial(6)
[0.00000064s] factorial(1) -> 1
[0.00004356s] factorial(2) -> 2
[0.00006712s] factorial(3) -> 6
[0.00009733s] factorial(4) -> 24
[0.00037927s] factorial(5) -> 120
[0.00043982s] factorial(6) -> 720


720

---
### Example 7-17: An imporved clock decorator

In [ ]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_list = []
        if args:
            arg_list.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ["%s=%s" % (k, v) for k, v in sorted(kwargs.items())]
            arg_list.append(', '.join(pairs))
        args_str = ', '.join(arg_list)
        print("[%0.8fs] %s(%s) -> %r" %(elapsed, name, args_str, result))
        return result

    return clocked

In [ ]:
@clock
def snooze(seconds, *args, **kwargs):
    time.sleep(seconds)

@clock
def factorial(n, *args, **kwargs):
    return 1 if n < 2 else n * factorial(n-1, *args, **kwargs)


print("*"*40, "calling snooze(.123, key='value')")
snooze(.123, key='value')
print("*"*40, "calling factorial(6, 45, a=1, b=2)")
factorial(6, 45, a=1, b=2)

**************************************** calling snooze(.123, key='value')
[0.12669492s] snooze(0.123, key=value) -> None
**************************************** calling factorial(6, 45, a=1, b=2)
[0.00000191s] factorial(1, 45, a=1, b=2) -> 1
[0.00009608s] factorial(2, 45, a=1, b=2) -> 2
[0.00016284s] factorial(3, 45, a=1, b=2) -> 6
[0.00021791s] factorial(4, 45, a=1, b=2) -> 24
[0.00030589s] factorial(5, 45, a=1, b=2) -> 120
[0.00036907s] factorial(6, 45, a=1, b=2) -> 720


720

## Memoization with *functools.lru_cache*

---
### Example 7-18: The very costly recursive way to compute the nth number in the Fibonacci series

In [ ]:
@clock
def fibonancci(n):
    if n < 2:
        return n
    return fibonancci(n-2) + fibonancci(n-1)

fibonancci(6)

[0.00000000s] fibonancci(0) -> 0
[0.00001407s] fibonancci(1) -> 1
[0.00105810s] fibonancci(2) -> 1
[0.00000000s] fibonancci(1) -> 1
[0.00000095s] fibonancci(0) -> 0
[0.00000095s] fibonancci(1) -> 1
[0.00026798s] fibonancci(2) -> 1
[0.00036097s] fibonancci(3) -> 2
[0.00160503s] fibonancci(4) -> 3
[0.00000095s] fibonancci(1) -> 1
[0.00000000s] fibonancci(0) -> 0
[0.00000000s] fibonancci(1) -> 1
[0.00004578s] fibonancci(2) -> 1
[0.00010610s] fibonancci(3) -> 2
[0.00000000s] fibonancci(0) -> 0
[0.00000095s] fibonancci(1) -> 1
[0.00010300s] fibonancci(2) -> 1
[0.00000000s] fibonancci(1) -> 1
[0.00000000s] fibonancci(0) -> 0
[0.00000000s] fibonancci(1) -> 1
[0.00003529s] fibonancci(2) -> 1
[0.00008702s] fibonancci(3) -> 2
[0.00054193s] fibonancci(4) -> 3
[0.00068378s] fibonancci(5) -> 5
[0.00240803s] fibonancci(6) -> 8


8

---
### Example 7-19: Faster implementation using caching

In [ ]:
import functools

@functools.lru_cache
@clock
def fibonancci(n):
    if n < 2:
        return n
    return fibonancci(n-2) + fibonancci(n-1)

fibonancci(6)

[0.00000000s] fibonancci(0) -> 0
[0.00000095s] fibonancci(1) -> 1
[0.00042796s] fibonancci(2) -> 1
[0.00000191s] fibonancci(3) -> 2
[0.00079703s] fibonancci(4) -> 3
[0.00000119s] fibonancci(5) -> 5
[0.00085235s] fibonancci(6) -> 8


8

In [ ]:
fibonancci(30)

832040

## Generic Functions with Single Dispatch

In [ ]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

---
### Example 7-20: htmlize generates HTML tailored to different objects types

In [ ]:
htmlize([1, 2, 3])

'<pre>[1, 2, 3]</pre>'

In [ ]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [ ]:
htmlize('Heimlich & Co.\n- a game')

'<pre>&#x27;Heimlich &amp; Co.\\n- a game&#x27;</pre>'

---
### Example 7-21: singledispatch creates a custom htmlize.register to bundle several functions into a generic function

In [ ]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text: str) -> str:
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)


@htmlize.register(numbers.Integral)
def _(number: numbers.Integral):
    return '<pre>{0} (0x{0:x})</pre>'.format(number)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n\t<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n\t<li>' + inner + '<\li>\n</ul>'

In [ ]:
print(htmlize({1, 2, 3}))
print(htmlize(abs))
print(htmlize('Heimlich &Co.\n- a game'))
print(htmlize(42))
print(htmlize([1, "two", {4, 5.7, None}, ['a', 'b', 'c']]))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<p>Heimlich &amp;Co.<br>
- a game</p>
<pre>42 (0x2a)</pre>
<ul>
	<li><pre>1 (0x1)</pre></li>
	<li><p>two</p></li>
	<li><pre>{None, 4, 5.7}</pre></li>
	<li><ul>
	<li><p>a</p></li>
	<li><p>b</p></li>
	<li><p>c</p><\li>
</ul><\li>
</ul>


## Parameterized Decorator

---
### Example 7-22: Abridged registration.py module from Example 7-2, repeated here for convenience

In [ ]:
registry = []

def register(func):
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')


print('running main()')
print('registry ->', registry)
f1()

running main()
registry -> [<function f1 at 0x10aab9d30>]
running f1()


## A parameterized Registration Decorator

---
### Example 7-23: To accept parameters, the new register decorator must be called as a function

In [ ]:
registry = set()

def register(active: bool=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)'
               % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)

        return func
        
    return decorate


@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')



running register(active=False)->decorate(<function f1 at 0x10aacb4c0>)
running register(active=True)->decorate(<function f2 at 0x10aab93a0>)


---
### Example 7-24: Using the registration decorator listed in example 7-23

In [ ]:
print(registry)

register()(f3)
print(registry)

register(active=False)(f2)
print(registry)

{<function f2 at 0x10aab93a0>}
running register(active=True)->decorate(<function f3 at 0x10aab9f70>)
{<function f2 at 0x10aab93a0>, <function f3 at 0x10aab9f70>}
running register(active=False)->decorate(<function f2 at 0x10aab93a0>)
{<function f3 at 0x10aab9f70>}


## The Parameterized Clock Decorator

---
### Example 7-25: Module clockdeco_param.py the parameterized clock decorator

In [ ]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate


@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(i)


[0.00001001s] snooze(0) -> None
[1.00079370s] snooze(1) -> None
[2.00021815s] snooze(2) -> None


---
### Example 7-26: clockdeco param demo 1

In [ ]:
import time
from clockdeco_param import clock

@clock('{name} : {elapsed} : {args}')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(i)

snooze : 9.059906005859375e-06 : 0
snooze : 1.0028400421142578 : 1
snooze : 2.004915952682495 : 2


---
### Example 7-27: clockdeco param demo 2

In [90]:
import time
from clockdeco_param import clock

@clock(fmt='{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.126s
snooze(0.123) dt=0.124s
snooze(0.123) dt=0.123s
